In [15]:
import matplotlib.pyplot as plt
import numpy as np
import os

from dotenv import load_dotenv
from dotenv import dotenv_values
from collections import namedtuple
import json
import yaml
import supervisely_lib as sly

secret_dotenv_file = '../secret_debug.env'
load_dotenv(secret_dotenv_file)
dotenv_file = '../debug.env'
load_dotenv(dotenv_file)

import sys
sys.path.append('../../')
from src.bounding_box import BoundingBox
from src.utils.enumerators import BBFormat, BBType, CoordinatesType
from src.evaluators.pascal_voc_evaluator import get_pascalvoc_metrics
from src.utils.enumerators import MethodAveragePrecision

In [16]:
api: sly.Api = sly.Api.from_env()
app: sly.AppService = sly.AppService()

In [17]:
task_id = os.environ['TASK_ID']
src_project_id = os.environ['modal.state.slySrcProjectId']
dst_project_id = os.environ['modal.state.slyDstProjectId']
src_project_id, dst_project_id

('2753', '2755')

In [18]:
src_project = app.public_api.project.get_info_by_id(src_project_id)
if src_project is None:
    raise RuntimeError(f"Project id={src_project_id} not found")
    
dst_project = app.public_api.project.get_info_by_id(dst_project_id)
if dst_project is None:
    raise RuntimeError(f"Project id={dst_project_id} not found")

In [19]:
result = namedtuple('Result', ['TP', 'FP', 'Precision', 'Recall', 'AP'])
def dict2tuple(dictionary, round_level = 4):
    FP = 0
    TP = 0
    npos = 0
    for dict_ in dictionary['per_class']:
        dict__ = dictionary['per_class'][dict_]
        FP += dict__['total FP']
        TP += dict__['total TP']
        npos += dict__['total positives']
    AP = round(dictionary['mAP'], round_level)
    Recall = round(TP / npos, round_level)
    Precision = round(np.divide(TP, (FP + TP)), round_level)
    return result(TP, FP, Precision, Recall, AP)


def plt2bb(batch_element,
           type_coordinates = CoordinatesType.ABSOLUTE, 
           bb_type          = BBType.GROUND_TRUTH, 
           format           = BBFormat.XYX2Y2):
    # type_coordinates = CoordinatesType.X : ABSOLUTE, RELATIVE
    # bb_type          = BBType.X          : GROUND_TRUTH, DETECTED
    # format           = BBFormat.X        : XYX2Y2, XYWH, PASCAL_XML, YOLO
    ret = []
    annotations = batch_element.annotation['objects']
    for ann in annotations:
        classTitle = ann['classTitle']
        points = ann['points']['exterior']
        x1,y1 = points[0]
        x2,y2 = points[1]
        confidence = None if bb_type == BBType.GROUND_TRUTH else ann['tags'][0]['value']
        bb = BoundingBox(image_name      = batch_element.image_name,
                         class_id        = classTitle,
                         coordinates     = (x1, y1, x2, y2),
                         type_coordinates= type_coordinates,
                         img_size        = (batch_element.annotation['size']['width'], 
                                            batch_element.annotation['size']['height']),
                         confidence      = confidence,
                         bb_type         = bb_type,
                         format          = format)
        ret.append(bb)
    return ret

In [1]:
import numpy as np

In [2]:
array = np.array([
    [1,'cat','jack'],
    [1,'dog','jim'],
    [2,'cat','tony'],
    [3,'dog','bill']
], dtype=object)

In [13]:
m = [el in [1,2] for el in array[:, 0]]

In [14]:
array[m]

array([[1, 'cat', 'jack'],
       [1, 'dog', 'jim'],
       [2, 'cat', 'tony']], dtype=object)

In [3]:
list(set(array[:,1]))

['cat', 'dog']

In [4]:
for name in set(array[:,1]):
    print(name)

cat
dog


In [5]:
import random

In [6]:
array[[1,3]]

array([[1, 'dog', 'jim'],
       [3, 'dog', 'bill']], dtype=object)

In [7]:
random.sample([1,2,3,4], 2)

[3, 4]

In [8]:
np.vstack([array[:2], array[2:]])

array([[1, 'cat', 'jack'],
       [1, 'dog', 'jim'],
       [2, 'cat', 'tony'],
       [3, 'dog', 'bill']], dtype=object)

array([[2, 'cat', 'tony'],
       [3, 'dog', 'bill']], dtype=object)

In [76]:
int(np.ceil(555/100 * 3))

17

In [ ]:
np.ro

In [55]:
len(array)

4

In [54]:
array[np.where(array[:, 0] == 1)]

array([[1, 'cat', 'jack'],
       [1, 'dog', 'jim']], dtype=object)

In [48]:
a = array[array[:,1]=='cat']

IndexError: boolean index did not match indexed array along dimension 1; dimension is 3 but corresponding boolean dimension is 4

In [36]:
for i in range(10):
    if i == 5 :
        continue
    print(i)

0
1
2
3
4
6
7
8
9


In [31]:
TEAM_ID = 7
data = get_data_v2(TEAM_ID, src_project, dst_project, batch_size=30)

Exists


In [20]:
import pickle

In [24]:
local_path = os.path.join(app.data_dir, 'src_dst_annotations.pkl')

In [25]:
with open(local_path, 'rb') as f:
    data = pickle.load(f)

In [30]:
set(data[:, 3])

{'train', 'trainval', 'val'}

In [31]:
TEAM_ID = 7
report_name = 'src_dst_annotations.pkl'
local_path = os.path.join(app.data_dir, report_name)
remote_path = f"/reports/mAP_Calculator/{dst_project.name}/{report_name}"

if api.file.exists(TEAM_ID, remote_path):
    api.file.download(TEAM_ID, remote_path, local_path)
    with open(local_path, 'rb') as f:
        data = pickle.load(f)

In [45]:
prj_ids = list(set(data[:, 0]))
src_prj_items = data[np.where(data[:, 0] == prj_ids[0])]
dst_prj_items = data[np.where(data[:, 0] == prj_ids[1])]

In [57]:
percentage = 10
selection_indexes = list()
src_list = list()
dst_list = list()

for name in set(src_prj_items[:, 3]):
    src_ds_items = src_prj_items[src_prj_items[:, 3] == name]
    dst_ds_items = dst_prj_items[dst_prj_items[:, 3] == name]
    length = len(src_ds_items)
    sample_size = int(np.ceil(length / 100 * percentage))
    indexes = random.sample(range(length), sample_size)
    src_list.extend(src_ds_items[indexes])
    dst_list.extend(dst_ds_items[indexes])
    
src_list = np.array(src_list)
dst_list = np.array(dst_list)
data = np.vstack([src_list, dst_list])
image_num = len(src_prj_items)

train
1464
1464
trainval
2913
2913
val
1449
1449


In [63]:
image_num

5826

In [62]:
data.shape

(1168, 8)

In [32]:
set(data[:, 3])

{'train', 'trainval', 'val'}

In [66]:
round_level = 4 # round(target_value, round_level): round(0.123456, round_level) --> 0.1235
ious = [0.5, 0.75] # list of IOU thresholds

# statistic data structure
struct = namedtuple('Set_mAP', ['name', 'mAP_05', 'mAP_075'])

# storage lists for images, datasets, projects
image_mAP   = [] # image_name   + mAP_05 + mAP_075
dataset_mAP = [] # dataset_name + mAP_05 + mAP_075
project_mAP = [] # project_name + mAP_05 + mAP_075

project_gts_bbs    = []
project_det_bbs    = []

for src, dst in zip(api.dataset.get_list(src_project.id), api.dataset.get_list(dst_project.id)):
    src_images = api.image.get_list(src.id)
    dst_images = api.image.get_list(dst.id)
    
    dataset_gts_bbs    = []
    dataset_det_bbs    = []
   
    for src_batch, dst_batch in zip(sly.batched(src_images, batch_size=10), 
                                    sly.batched(dst_images, batch_size=10)):
        src_image_ids   = [image_info.id   for image_info in src_batch]
        src_image_names = [image_info.name for image_info in src_batch]
        dst_image_ids   = [image_info.id   for image_info in dst_batch]
        dst_image_names = [image_info.name for image_info in dst_batch]
        
        src_annotations = api.annotation.download_batch(src.id, src_image_ids)
        dst_annotations = api.annotation.download_batch(dst.id, dst_image_ids)
        assert len(src_annotations)==len(dst_annotations), \
                            'Lenghst of src_annotations and dst_annotations must be the same!'
        for src_annotation, dst_annotation in zip(src_annotations, dst_annotations):
            img_gts_bbs = plt2bb(src_annotation)
            img_det_bbs = plt2bb(dst_annotation, bb_type = BBType.DETECTED)
            dataset_gts_bbs.extend(img_gts_bbs)
            dataset_det_bbs.extend(img_det_bbs):
            image_mAPs = []
            for iou in ious:
                dict_res = get_pascalvoc_metrics(
                            img_gts_bbs, img_det_bbs, iou, generate_table=True, 
                            method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
                )
                image_mAPs.append(dict_res)
            print('image_mAPs______- =', image_mAPs)
            image_mAP.append(
                struct(
                    name    = src_annotation.image_name, 
                    mAP_05  = dict2tuple(image_mAPs[0], round_level), 
                    mAP_075 = dict2tuple(image_mAPs[1], round_level)
                )
            )
    
    project_gts_bbs.extend(dataset_gts_bbs)
    project_det_bbs.extend(dataset_det_bbs)
    
    mAPs = []
    for iou in ious:
        dict_res = get_pascalvoc_metrics(
                    dataset_gts_bbs, dataset_det_bbs, iou, generate_table=True, 
                    method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
        )
        mAPs.append(dict_res)
        
    dataset_mAP.append(
        struct(
            name    = src.name, 
            mAP_05  = dict2tuple(mAPs[0], round_level), 
            mAP_075 = dict2tuple(mAPs[1], round_level)
        )
    )

mAPs = []
for iou in ious:
    dict_res = get_pascalvoc_metrics(
                project_gts_bbs, project_det_bbs, iou, generate_table=True, 
                method=MethodAveragePrecision.EVERY_POINT_INTERPOLATION
    )
    mAPs.append(dict_res)
    
project_mAP.append(
    struct(
        name    = src_project.name, 
        mAP_05  = dict2tuple(mAPs[0], round_level), 
        mAP_075 = dict2tuple(mAPs[1], round_level)
    )
)

image_mAPs______- = [{'per_class': {'cow': {'precision': array([1.]), 'recall': array([1.]), 'AP': 1.0, 'interpolated precision': [1.0, 1.0], 'interpolated recall': [0, 1.0], 'total positives': 1, 'total TP': 1.0, 'total FP': 0.0, 'method': <MethodAveragePrecision.EVERY_POINT_INTERPOLATION: 1>, 'iou': 0.5, 'table':              image confidence  TP  FP  acc TP  acc FP  precision  recall
0  2009_000250.jpg     90.43%   1   0     1.0     0.0        1.0     1.0}}, 'mAP': 1.0}, {'per_class': {'cow': {'precision': array([1.]), 'recall': array([1.]), 'AP': 1.0, 'interpolated precision': [1.0, 1.0], 'interpolated recall': [0, 1.0], 'total positives': 1, 'total TP': 1.0, 'total FP': 0.0, 'method': <MethodAveragePrecision.EVERY_POINT_INTERPOLATION: 1>, 'iou': 0.75, 'table':              image confidence  TP  FP  acc TP  acc FP  precision  recall
0  2009_000250.jpg     90.43%   1   0     1.0     0.0        1.0     1.0}}, 'mAP': 1.0}]
image_mAPs______- = [None, None]


TypeError: 'NoneType' object is not subscriptable

In [69]:
src_project.id

2753

In [73]:
datasets = api.dataset.get_list(src_project.id)
for dataset in datasets:
    image_list = api.image.get_list(dataset.id)
    print(len(image_list))

2913
1449
1464


In [81]:
import random

In [87]:
random.sample(image_list, 10)

[ImageInfo(id=815871, name='2008_006389.jpg', link=None, hash='Jf6xb1rbLGQH12jq0TEojnfDGnqidws6rm8TdUs4sME=', mime='image/jpeg', ext='jpeg', size=163770, width=500, height=375, labels_count=0, dataset_id=4214, created_at='2021-04-20T07:53:29.915Z', updated_at='2021-04-20T09:10:39.267Z', meta={}, path_original='/h5un6l2bnaz1vj8a9qgms4-public/images/original/F/Q/lC/hgR91bGlrEMetXwFUzVNuoiRMXL7fYXWADo8XCHzPLypZWpBXCix2Q9v7ornVlPek9pG58nnf5IZolrpyl4dg3PYCd9ja0x5GcWaPZl1gXXOc4DM8RvGhElK8Q4b.jpg', full_storage_url='http://78.46.75.100:38585/h5un6l2bnaz1vj8a9qgms4-public/images/original/F/Q/lC/hgR91bGlrEMetXwFUzVNuoiRMXL7fYXWADo8XCHzPLypZWpBXCix2Q9v7ornVlPek9pG58nnf5IZolrpyl4dg3PYCd9ja0x5GcWaPZl1gXXOc4DM8RvGhElK8Q4b.jpg'),
 ImageInfo(id=816257, name='2008_006908.jpg', link=None, hash='cDSkqSA/UL/wPqpjFIPu/U4iEI3fDQkA2L30YE4rMKc=', mime='image/jpeg', ext='jpeg', size=139045, width=500, height=332, labels_count=1, dataset_id=4214, created_at='2021-04-20T07:53:32.417Z', updated_at='2021-04-20T09

In [80]:
image_list

[ImageInfo(id=815780, name='2009_000250.jpg', link=None, hash='D1qOyWOEHEnWEo2GvGKVE6FUs4iQ0BGUqiCJ5DZVF7Q=', mime='image/jpeg', ext='jpeg', size=194832, width=500, height=333, labels_count=1, dataset_id=4214, created_at='2021-04-20T07:53:29.638Z', updated_at='2021-04-20T09:10:39.267Z', meta={}, path_original='/h5un6l2bnaz1vj8a9qgms4-public/images/original/5/f/7O/IFvdR4krtBnKYTejii2K8LoYHryBxBSpSL95vaT1MztuIxsaVlXl7aoEi0tGY4ZY3I4MUWbUWOPrdSeStk6G95QDczidaE5a8pnujBV76T8jtivY7nmVuY9xaOAi.jpg', full_storage_url='http://78.46.75.100:38585/h5un6l2bnaz1vj8a9qgms4-public/images/original/5/f/7O/IFvdR4krtBnKYTejii2K8LoYHryBxBSpSL95vaT1MztuIxsaVlXl7aoEi0tGY4ZY3I4MUWbUWOPrdSeStk6G95QDczidaE5a8pnujBV76T8jtivY7nmVuY9xaOAi.jpg'),
 ImageInfo(id=815781, name='2008_006751.jpg', link=None, hash='LV55EAm3pCFkkxZ+jZgjekCx28ht5VaahFVurialxKU=', mime='image/jpeg', ext='jpeg', size=172331, width=500, height=375, labels_count=0, dataset_id=4214, created_at='2021-04-20T07:53:29.638Z', updated_at='2021-04-20T09

In [ ]:
    for key, project in projects.items():
        bb_type = BBType.GROUND_TRUTH if key == 'src' else BBType.DETECTED
        datasets = api.dataset.get_list(project.id)
        for dataset in datasets:
            image_list = api.image.get_list(dataset.id)

In [77]:
r = np.array([[1,2,3],[1,2,3],[1,2,3]])
r

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [79]:
1 in set(r[:, 1])

False

In [65]:
dict_res

In [ ]:
project_mAP

In [ ]:
dataset_mAP

In [ ]:
image_mAP

In [ ]:
# build final table
pd_data = []
columns = ["Image Nmae", "IOU05_TP",  "IOU05_FP",  "IOU05_Precision",  "IOU05_Recall",  "IOU05_AP", 
                         "IOU075_TP", "IOU075_FP", "IOU075_Precision", "IOU075_Recall", "IOU075_AP"]

In [ ]:
pd_data = []
for element in image_mAP:
    element_data = []
    element_data.append(element.name)
    for el in element[1:]:
        element_data.extend(el)
    pd_data.append(element_data)

In [ ]:
pd_data

In [ ]:
for i in image_mAP[0]:
    print(i)

In [ ]:
image_mAP[0][1:]

In [ ]:
a = []

In [ ]:
a.extend(i)

In [ ]:
a

In [ ]:
d = {
    'a':1,
    'b':2
}

In [ ]:
for k,v in d.items():
    print(k,v)